# Prerequisite Code

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



# Hyperparameters

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Optimization Loop

Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an epoch.

Each epoch consists of two main parts:
*   The Train Loop - iterate over the training dataset and try to converge to optimal parameters.
*   The Validation/Test Loop - iterate over the test dataset to check if model performance is improving.




# Loss Function

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Full Implementation

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.314390  [    0/60000]
loss: 2.292752  [ 6400/60000]
loss: 2.282062  [12800/60000]
loss: 2.271103  [19200/60000]
loss: 2.249671  [25600/60000]
loss: 2.232169  [32000/60000]
loss: 2.226171  [38400/60000]
loss: 2.201542  [44800/60000]
loss: 2.195776  [51200/60000]
loss: 2.174767  [57600/60000]
Test Error: 
 Accuracy: 53.3%, Avg loss: 2.164791 

Epoch 2
-------------------------------
loss: 2.176509  [    0/60000]
loss: 2.162058  [ 6400/60000]
loss: 2.116547  [12800/60000]
loss: 2.127064  [19200/60000]
loss: 2.075230  [25600/60000]
loss: 2.022413  [32000/60000]
loss: 2.039232  [38400/60000]
loss: 1.971497  [44800/60000]
loss: 1.968109  [51200/60000]
loss: 1.903872  [57600/60000]
Test Error: 
 Accuracy: 56.7%, Avg loss: 1.906153 

Epoch 3
-------------------------------
loss: 1.935702  [    0/60000]
loss: 1.907364  [ 6400/60000]
loss: 1.804625  [12800/60000]
loss: 1.836790  [19200/60000]
loss: 1.723690  [25600/60000]
loss: 1.671856  [32000/600